# RQ2. How does interpretability impacts training?

1. Occlude high-ranked RoIs, train and measure the performance drop
2. Train ML from saliency dict, train and measure their performance drop

In [1]:
import matplotlib.pyplot as plt

import weight_parser as wp
import stats as s
import constants as C
import plotting as p
import metadata as m
import utils as u

### 1. Occlusion Training